In [7]:
from langchain.chat_models import BedrockChat
from langchain.schema import HumanMessage
from langchain.tools import YouTubeSearchTool
from langchain.agents import load_tools
import os
import boto3

# os.environ['AWS_ACCESS_KEY_ID'] = ""
# os.environ['AWS_SECRET_ACCESS_KEY'] = ""

BEDROCK_CLIENT = boto3.client("bedrock-runtime", 'us-east-1')

llm = BedrockChat(model_id="anthropic.claude-v2:1", model_kwargs={"temperature": 0}, client=BEDROCK_CLIENT)

tools = [YouTubeSearchTool()]

#tools = load_tools(["requests_all"])

In [8]:
prompt_template = """
You are a helfull assistant, your task is help the user achieve is goal.
In this environment you have access to a set of tools you can use to answer the user's question.
Use them in a smart way to fullfill our objetives. Always consider the history of previous steps to ensure a continuos progress towards the objetive.
When you have the answer for the user, always answer it in this forma bellow:
Final Answer: answer to user


You may call them like this. Only invoke one function at a time and wait for the results before invoking another function:
<function_calls>
<invoke>
<tool_name>$TOOL_NAME</tool_name>
<parameters>
<$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
...
</parameters>
</invoke>
</function_calls>

Here are the tools available:
<tools>
{tools_string}
</tools>


User:
{input}

"""

In [9]:
from langchain_core.tools import BaseTool

from langchain_community.utils.openai_functions import (
    FunctionDescription,
    ToolDescription,
    convert_pydantic_to_openai_function,
)

def format_tool_to_anthropic_function(tool: BaseTool) -> FunctionDescription:
	"""Format tool into the Anthropic function API."""
	new_tool = f"""
<tool_description>
<tool_name>{tool.name}</tool_name>
<description>
{tool.description}
<parameters>
<parameter>
<name>__arg1</name>
<type>string</type>
<description>The input of the function</description>
</parameter>
</parameters>"""
	return new_tool

In [10]:
import json
from json import JSONDecodeError
from typing import List, Union

import xml.etree.ElementTree as ET

from langchain_core.agents import AgentAction, AgentActionMessageLog, AgentFinish
from langchain_core.exceptions import OutputParserException
from langchain_core.messages import (
    AIMessage,
    BaseMessage,
)
from langchain_core.outputs import ChatGeneration, Generation

from langchain.agents.agent import AgentOutputParser


class AnthropicFunctionsAgentOutputParser(AgentOutputParser):

    @staticmethod
    def _extract_function_call(full_text):
        # Encontrar o início e o fim da string XML
        if "function_calls" not in full_text:
            return None

        start = full_text.find("<function_calls>")
        end = full_text.find("</function_calls>") + len("</function_calls>")

        # Extrair a string XML
        xml_string = full_text[start:end]

        # Análise da string XML
        root = ET.fromstring(xml_string)

        # Encontrar o nome da ferramenta
        tool_name = root.find(".//tool_name").text

        # Construir o dicionário de parâmetros
        parameters = {}
        for param in root.findall(".//parameters/*"):
            parameters[param.tag] = param.text

        # Construir o dicionário final com nome e parâmetros
        return {
            "name": tool_name,
            "arguments": parameters
        }

    @property
    def _type(self) -> str:
        return "openai-functions-agent"

    @staticmethod
    def _parse_ai_message(message: BaseMessage) -> Union[AgentAction, AgentFinish]:
        """Parse an AI message."""
        if not isinstance(message, AIMessage):
            raise TypeError(f"Expected an AI message got {type(message)}")

        function_call = AnthropicFunctionsAgentOutputParser._extract_function_call(message.content)

        if function_call and 'Final Answer:' not in message.content:
            function_name = function_call["name"]
            try:
                if len(function_call.keys()) == 0:
                    # OpenAI returns an empty string for functions containing no args
                    _tool_input = {}
                else:
                    # otherwise it returns a json object
                    _tool_input = function_call["arguments"]
            except JSONDecodeError:
                raise OutputParserException(
                    f"Could not parse tool input: {function_call} because "
                    f"the `arguments` is not valid JSON."
                )

            if "__arg1" in _tool_input.keys():
                tool_input = _tool_input["__arg1"]
            else:
                tool_input = _tool_input

            content_msg = f"responded: {message.content}\n" if message.content else "\n"
            log = f"\nInvoking: `{function_name}` with `{tool_input}`\n{content_msg}\n"
            return AgentActionMessageLog(
                tool=function_name,
                tool_input=tool_input,
                log=log,
                message_log=[message],
            )

        start = message.content.find("<function_calls>")

        return AgentFinish(
            return_values={"output": message.content[start:]}, log=str(message.content)
        )

    def parse_result(
            self, result: List[Generation], *, partial: bool = False
        ) -> Union[AgentAction, AgentFinish]:
            if not isinstance(result[0], ChatGeneration):
                raise ValueError("This output parser only works on ChatGeneration output")
            message = result[0].message
            return self._parse_ai_message(message)


    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        raise ValueError("Can only parse messages")


In [11]:
import json
from typing import List, Sequence, Tuple

from langchain_core.agents import AgentAction, AgentActionMessageLog
from langchain_core.messages import AIMessage, BaseMessage, FunctionMessage

def _convert_agent_action_to_messages(agent_action: AgentAction, observation: str) -> List[BaseMessage]:
    """Convert an agent action to a message.

    This code is used to reconstruct the original AI message from the agent action.

    Args:
        agent_action: Agent action to convert.

    Returns:
        AIMessage that corresponds to the original tool invocation.
    """
    if isinstance(agent_action, AgentActionMessageLog):
        return str(list(agent_action.message_log) + [_create_function_message(agent_action, observation)])
    else:
        return agent_action.log

def _create_function_message(agent_action: AgentAction, observation: str) -> FunctionMessage:
    """Convert agent action and observation into a function message.
    Args:
        agent_action: the tool invocation request from the agent
        observation: the result of the tool invocation
    Returns:
        FunctionMessage that corresponds to the original tool invocation
    """
    if not isinstance(observation, str):
        try:
            content = json.dumps(observation, ensure_ascii=False)
        except Exception:
            content = str(observation)
    else:
        content = observation
    return agent_action.tool+":"+content


def format_to_openai_function_messages(intermediate_steps: Sequence[Tuple[AgentAction, str]]) -> List[BaseMessage]:
    """Convert (AgentAction, tool output) tuples into FunctionMessages.

    Args:
        intermediate_steps: Steps the LLM has taken to date, along with observations

    Returns:
        list of messages to send to the LLM for the next prediction
    """
    messages = ""

    for agent_action, observation in intermediate_steps:
        messages.extend(_convert_agent_action_to_messages(agent_action, observation))

    return messages


In [12]:
from langchain_core.runnables import Runnable, RunnablePassthrough
from langchain.agents.format_scratchpad.openai_functions import (
    format_to_openai_function_messages,
)
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentExecutor

prompt = ChatPromptTemplate.from_messages(
    [
        ("human", prompt_template+ "History: {agent_scratchpad}"),

    ]
)


agent = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        )
    )
    | prompt
    | llm
    | AnthropicFunctionsAgentOutputParser()
)

chain = prompt | llm.bind(stop_sequences=['Final Answer:'])

tools_string = "<tools>"+"".join([format_tool_to_anthropic_function(t) for t in tools])+"</tools>"

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input":"List 5 youtube videos that talk about AGI usando a tool disponível", "tools_string":f"<tools>{tools_string}</tools>"})



> Entering new AgentExecutor chain...



Invoking: `youtube_search` with `AGI, 5`
responded:  <function_calls>
<invoke>
<tool_name>youtube_search</tool_name>
<parameters>
<__arg1>AGI, 5</__arg1>
</parameters>
</invoke>
</function_calls>

['https://www.youtube.com/watch?v=2gc3ZnZTY8E&pp=ygUDQUdJ', 'https://www.youtube.com/watch?v=D1nc80yMWqA&pp=ygUDQUdJ', 'https://www.youtube.com/watch?v=JUX24r36OLU&pp=ygUDQUdJ', 'https://www.youtube.com/watch?v=ICgjWMZp88U&pp=ygUDQUdJ', 'https://www.youtube.com/watch?v=tPul4xH2GAo&pp=ygUDQUdJ']
Invoking: `youtube_search` with `AGI, 5`
responded:  <function_calls>
<invoke>
<tool_name>youtube_search</tool_name>
<parameters>
<__arg1>AGI, 5</__arg1>
</parameters>
</invoke>
</function_calls>

['https://www.youtube.com/watch?v=2gc3ZnZTY8E&pp=ygUDQUdJ', 'https://www.youtube.com/watch?v=D1nc80yMWqA&pp=ygUDQUdJ', 'https://www.youtube.com/watch?v=JUX24r36OLU&pp=ygUDQUdJ', 'https://www.youtube.com/watch?v=ICgjWMZp88U&pp=ygUDQUdJ', 'https://www.youtube.com/watch?v=tPul4xH2GAo&pp=ygUDQUdJ'] <function_cal

{'input': 'List 5 youtube videos that talk about AGI usando a tool disponível',
 'tools_string': '<tools><tools>\n<tool_description>\n<tool_name>youtube_search</tool_name>\n<description>\nsearch for youtube videos associated with a person. the input to this tool should be a comma separated list, the first part contains a person name and the second a number that is the maximum number of video results to return aka num_results. the second part is optional\n<parameters>\n<parameter>\n<name>__arg1</name>\n<type>string</type>\n<description>The input of the function</description>\n</parameter>\n</parameters></tools></tools>',
 'output': "<function_calls>\n<invoke>\n<tool_name>youtube_search</tool_name>\n<parameters>\n<__arg1>AGI, 5</__arg1>\n</parameters>\n</invoke>\n</function_calls>\n\nFinal Answer: ['https://www.youtube.com/watch?v=2gc3ZnZTY8E&pp=ygUDQUdJ', 'https://www.youtube.com/watch?v=D1nc80yMWqA&pp=ygUDQUdJ', 'https://www.youtube.com/watch?v=JUX24r36OLU&pp=ygUDQUdJ', 'https://www.yo